In [1]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
### Tavily Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
tavily = TavilySearchResults()
# tavily.invoke("Provide me the recent AI news for June 14, 2025")


In [3]:
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
class ClassTemplates:
    TOOLCALL_TEMPLATE = """
    """

    CLASSIFIER_PROMPT = """
        Analyze the user's financial query and generate a JSON response with the appropriate task type and action plan.

        ### Available Data:
        - Fields: {available_fields}
        - Years: {available_years}

        ### Task Types:
        1. "retrieve_numbers": When the query asks for specific numeric values
        2. "perform_calculations": When the query requires mathematical operations
        3. "give_advice": When the query seeks recommendations or analysis
        4. "other": For all other cases

        ### Response Structure Rules:
        1. For "retrieve_numbers":
        - Include an "items" list with format: ["FieldName, Year"]
        - Always use exact field names from available_fields
        - Always use years from available_years

        2. For "perform_calculations":
        - Create a "plan" object with numbered steps
        - Each step must be one of:
            * retrieve ["FieldName, Year"]
            * add ["stepX", "stepY"]
            * subtract ["stepX", "stepY"] 
            * multiply ["stepX", "stepY"]
            * divide ["stepX", "stepY"]
            * return_percentage ["stepX", "stepY"]

        3. For "give_advice":
        - Provide a "description" of the advice needed

        ### Special Handling:
        - These terms ALWAYS indicate calculations: sum, total, net, ratio, percentage, per, rate, cumulative, combined, difference, overall, growth, change
        - Always validate field names and years against available data
        - For ambiguous requests, default to retrieval

        ### Examples:

        [Example 1: Simple Retrieval]
        Query: "What was revenue in 2023?"
        {{
        "task_type": "retrieve_numbers",
        "items": ["Revenue, 2023"]
        }}

        [Example 2: Compound Calculation]
        Query: "What's the profit margin for 2022?"
        {{
        "task_type": "perform_calculations",
        "plan": {{
            "step1": {{"action": "retrieve", "args": ["Net Income", "2022"]}},
            "step2": {{"action": "retrieve", "args": ["Revenue", "2022"]}},
            "step3": {{"action": "return_percentage", "args": ["step1", "step2"]}}
        }}
        }}

        [Example 3: Multiple Retrievals]
        Query: "Show me capital and expenses for 2021-2023"
        {{
        "task_type": "retrieve_numbers",
        "items": [
            "Capital, 2021",
            "Expenses, 2021",
            "Capital, 2022",
            "Expenses, 2022",
            "Capital, 2023",
            "Expenses, 2023"
        ]
        }}

        [Example 4: Complex Calculation]
        Query: "What's the ratio of R&D to Marketing in 2022?"
        {{
        "task_type": "perform_calculations",
        "plan": {{
            "step1": {{"action": "retrieve", "args": ["R&D Expense", "2022"]}},
            "step2": {{"action": "retrieve", "args": ["Marketing Expense", "2022"]}},
            "step3": {{"action": "divide", "args": ["step1", "step2"]}}
        }}
        }}

        [Example 5: Advice Request]
        Query: "How can we reduce operational costs?"
        {{
        "task_type": "give_advice",
        "description": "strategies for reducing operational costs"
        }}

        ### Current Query:
        {query}

        Respond ONLY with valid JSON matching one of the above formats.
        """

In [ ]:
@staticmethod
def clean_dataframe(df):
    """
    Cleans a dataframe by:
    1. Identifying the row that contains year values (both integer and float).
    2. Renaming columns using the detected years.
    3. Removing metadata rows above the detected year row.
    4. Ensuring all columns have valid names.
    5. Filling NaN values with -inf.
    
    Parameters:
    df (pd.DataFrame): Raw dataframe with metadata and financial data.
    
    Returns:
    pd.DataFrame: Processed dataframe with correct column names and missing values replaced.
    """
    
    # Identify the row index where the first numeric year appears
    year_row_index = None
    for i in range(len(df)):
        non_null_values = df.iloc[i].dropna()
        if non_null_values.astype(str).str.match(r'^\d{4}(\.0)?$').all():  # Matches years like 2024 and 2024.0
            year_row_index = i
            break

    if year_row_index is None:
        raise ValueError("No year row found in the dataset.")

    # Extract column names from the identified row (convert floats to int for clean column names)
    new_columns = df.iloc[year_row_index+1].values.astype(str).tolist()
    new_columns = [col if not col.replace('.0', '').isdigit() else str(int(float(col))) for col in new_columns]

    # Ensure the first column has a proper name (e.g., "Category")
    new_columns[0] = "category"

    # Remove metadata rows above the detected year row and reset index
    df = df.iloc[year_row_index+2:,]

    # Assign new column names
    df.columns = new_columns # 

    # Fill NaN values with -inf
    df = df.fillna(-np.inf).reset_index(drop=True, inplace=False)
    df.set_index('category', inplace=True)
    
    return df


if __name__ == "__main__":
# For now, just limited to 1 sheet
excelPandas = pd.read_excel("../../example_sheets/Detailed_Expense_Breakdown.xlsx", sheet_name='Sheet1')
ExcelService.processExcel(excelPandas)